In [0]:
silver_path = "abfss://silver@livbusdatastore.dfs.core.windows.net/"

ServicePrincipalId = dbutils.secrets.get(scope="livbodsbus-keyvault",key="dbx-client-ID")
ServicePrincipalKey = dbutils.secrets.get(scope="livbodsbus-keyvault",key="dbx-secret")
TenantId = dbutils.secrets.get(scope="livbodsbus-keyvault",key="dbx-tenant-ID")


# Configure access to ADLS Gen2
spark.conf.set("fs.azure.account.auth.type.livbusdatastore.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.livbusdatastore.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.livbusdatastore.dfs.core.windows.net", ServicePrincipalId)
spark.conf.set("fs.azure.account.oauth2.client.secret.livbusdatastore.dfs.core.windows.net", ServicePrincipalKey)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.livbusdatastore.dfs.core.windows.net", f"https://login.microsoftonline.com/{TenantId}/oauth2/token")

**Update previous day gold bus_activity to be used for analysis**

In [0]:
from pyspark.sql.functions import col, lag, when, unix_timestamp, to_date, sum, year, month, dayofmonth, date_sub, current_date, row_number
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# 1. Set dates
yesterday_date = date_sub(current_date(), 1)
day_before_yesterday = date_sub(current_date(), 2)

yesterday_year = year(yesterday_date)
yesterday_month = month(yesterday_date)
yesterday_day = dayofmonth(yesterday_date)

# 2. Load yesterday's data using partition pruning
df_yesterday = spark.read.format("delta").table("silver.bus_activity") \
    .filter(
        (col("year") == yesterday_year) &
        (col("month") == yesterday_month) &
        (col("day") == yesterday_day)
    )

# 3. Load last record per vehicle from day before yesterday
df_prev = spark.read.format("delta").table("silver.bus_activity") \
    .filter(
        (col("year") == year(day_before_yesterday)) &
        (col("month") == month(day_before_yesterday)) &
        (col("day") == dayofmonth(day_before_yesterday))
    )

window_desc = Window.partitionBy("vehicle_ref").orderBy(col("ingestion_timestamp").desc())
df_prev_latest = df_prev.withColumn("rn", row_number().over(window_desc)) \
                        .filter(col("rn") == 1).drop("rn")

# 4. Combine both datasets
df = df_yesterday.unionByName(df_prev_latest)

# 5. Define Liverpool bounding box
liverpool_min_lat, liverpool_max_lat = 53.33, 53.50
liverpool_min_lon, liverpool_max_lon = -3.00, -2.86

# 6. Add 'in_liverpool' column
df = df.withColumn(
    "in_liverpool",
    when(
        (col("latitude").between(liverpool_min_lat, liverpool_max_lat)) &
        (col("longitude").between(liverpool_min_lon, liverpool_max_lon)),
        True
    ).otherwise(False)
)

# 7. Define window by vehicle and time
vehicle_window = Window.partitionBy("vehicle_ref").orderBy("ingestion_timestamp")

# 8. Lag previous values
df = df.withColumn("prev_latitude", lag("latitude").over(vehicle_window)) \
       .withColumn("prev_longitude", lag("longitude").over(vehicle_window)) \
       .withColumn("prev_in_liverpool", lag("in_liverpool").over(vehicle_window)) \
       .withColumn("prev_recorded_time", lag("recorded_at_time").over(vehicle_window))

# 9. Calculate duration in minutes
df = df.withColumn(
    "dur_min_since_last_recorded",
    (unix_timestamp("recorded_at_time") - unix_timestamp("prev_recorded_time")) / 60
)

# 10. Detect idle state
df = df.withColumn(
    "possibly_idle",
    when(
        ((col("latitude") == col("prev_latitude")) & 
         (col("longitude") == col("prev_longitude"))) |
         (col("recorded_at_time") == col("prev_recorded_time")),
        True
    ).otherwise(False)
)

# 11. Label Liverpool movement
df = df.withColumn(
    "liverpool_movement_status",
    when((col("prev_in_liverpool").isNull()) & (col("in_liverpool") == True), "Only_In_Liverpool")
    .when(col("prev_in_liverpool") != col("in_liverpool"), "Moved_In_Or_Out")
    .otherwise("Other")
)

# 12. Infer routes that visit Liverpool
liverpool_routes = df.filter(col("in_liverpool") == True) \
                     .select("line_ref", "operator_ref") \
                     .distinct()

# 13. Join back to get data for Liverpool-related routes
df_liverpool_routes = df.join(liverpool_routes, on=["line_ref", "operator_ref"], how="inner")

# 14. Apply filter rules
bus_rules_path = "abfss://config@livbusdatastore.dfs.core.windows.net/bus_line_rules.json"
line_rules = spark.read.option("multiline", "true").json(bus_rules_path).collect()[0]

lines_to_remove = line_rules["lines_to_exclude_completely"]

df_final = df_liverpool_routes.filter(~col("line_ref").isin(lines_to_remove))

for rule in line_rules["special_filters"]:
    lines = rule["lines"]
    lat_condition = rule["latitude_condition"]
    lat_value = rule["latitude_value"]
    
    if lat_condition == ">":
        df_final = df_final.filter(
            ~(
                (col("line_ref").isin(lines)) & (col("latitude") > lat_value)
            )
        )
    elif lat_condition == "<":
        df_final = df_final.filter(
            ~(
                (col("line_ref").isin(lines)) & (col("latitude") < lat_value)
            )
        )

# 15. Add ingestion_date
df_bus_activity = df_final.withColumn("ingestion_date", to_date(col("ingestion_timestamp")))

# 16. Remove duplicates for yesterday
bus_activity_path = "abfss://gold@livbusdatastore.dfs.core.windows.net/bus_activity"

if DeltaTable.isDeltaTable(spark, bus_activity_path):
    gold_table = DeltaTable.forPath(spark, bus_activity_path)
    gold_table.delete(f"ingestion_date = date_sub(current_date(), 1)")

# 17. Write enriched data to Gold
df_bus_activity.write.format("delta") \
    .mode("append") \
    .partitionBy("ingestion_date") \
    .save(bus_activity_path)

**Update bus trip counts in gold table**

In [0]:
from pyspark.sql.functions import col, lag, when, unix_timestamp, sum, to_date, date_sub, current_date
from pyspark.sql.window import Window

# Step 1: Get yesterday's date
yesterday_date = date_sub(current_date(), 1)

# Step 2: Read ONLY yesterday's data from gold.bus_activity
df = (
    spark.read.format("delta")
    .table("gold.bus_activity")
    .filter(col("ingestion_date") == yesterday_date)
)

# Step 3: Define the window
trip_window = Window.partitionBy("vehicle_ref", "line_ref", "ingestion_date").orderBy("ingestion_timestamp")

# Step 4: Lag previous direction
df = df.withColumn("prev_direction_ref", lag("direction_ref").over(trip_window))

# Step 5: Detect direction changes
df = df.withColumn(
    "trip_start",
    (col("direction_ref") != col("prev_direction_ref")).cast("int")  # 1 if direction changed, else 0
)

# Step 6: Sum the trip_start flags per bus per line per day
df_trip_counts = df.groupBy("vehicle_ref", "line_ref", "ingestion_date", "operator_ref") \
    .agg(
        sum("trip_start").alias("trip_count")
    )

# Step 7: Load the gold trip_counts table (assuming you already created it)
bus_trips_path = "abfss://gold@livbusdatastore.dfs.core.windows.net/bus_trips_counts"

# If not already loaded
trip_counts_table = DeltaTable.forPath(spark, bus_trips_path)

# Step 8: Delete old data for yesterday (if it exists)
trip_counts_table.delete("ingestion_date = date_sub(current_date(), 1)")

# Step 9: Append the new trip_counts
df_trip_counts.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .save(bus_trips_path)